<a href="https://colab.research.google.com/github/KOMBOU12/Marius/blob/main/M2_MALIASH_LLM_%2B_RAG_mo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot basé sur un modèle de langue pré-entraîné assisté par une base documentaire (RAG)

**Auteur :** Adrien Guille (Université Lumière Lyon 2), pour le cours *Representation learning for NLP* @ Master 2 MALIA.

## Préparation de l'envrionnement

1. Se connecter ou se créer un compte sur https://huggingface.co/
2. Se créer un nouveau token d'accès : https://huggingface.co/settings/tokens
3. Enregistrer ce token comme un secret nommé `HF_TOKEN` dans Google Colab (icone "clef" dans le bandeau vertical)
4. Demander l'accès aux modèles pré-entraînés par Meta (la série des modèles Llama 3) : https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct
5. Exécuter le code ci-dessous

In [ ]:
from google.colab import userdata
from huggingface_hub import login; login(token=userdata.get("HF_TOKEN"))

from nltk import download; download('punkt_tab')
!pip install -U bitsandbytes txtai

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Chargement du modèle Llama 3.2 3B de base

1. Instancier le tokenizer pour le modèle `meta-llama/Llama-3.2-3B` via la classe `AutoTokenizer`
2. Charger le modèle pré-entraîné de base via la classe AutoModelForCausalLM
3. Exécuter dans la deuxième cellule ci-dessous la commande `!nvidia-smi` pour consulter la quantité de mémoire utilisée par le GPU
4. Modifier la cellule directement ci-dessous en ajoutant l'argument `torch_dtype=torch.float16` lors du chargement du modèle puis consulter de nouveau l'usage de la mémoire du GPU


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Instancier le tokenizer pour le modèle Llama-3.2-3B
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B")

# Charger le modèle pré-entraîné pour génération de texte
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B",
                                             return_dict=True,
                                             torch_dtype=torch.float16,
                                             device_map="auto")



tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
!nvidia-smi #afficher des informations sur les GPU NVIDIA disponibles sur la machine/cas du float d'origine :

Thu Dec 12 09:02:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0              27W /  70W |  14249MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!nvidia-smi #afficher des informations sur les GPU NVIDIA disponibles sur la machine/cas du float 16 :

Thu Dec 12 09:29:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0              27W /  70W |   6277MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Génération avec le modèle de base

1. Tokenizer la question pour obtenir la séquence de tokens (plus exactement la séquence d'identifiants des tokens), et envoyer le résultat sur le GPU
2. Générer la réponse (qui est aussi une séquence d'identifiants de tokens)
3. Convertir les identifiants en les tokens qu'ils désignent

In [ ]:
question = "Donne moi une bonne recette de pâtes bolognaises"
input_ids = tokenizer(question, add_special_tokens=True, return_tensors="pt").to("cuda")
#to("cpu") pour la RAM, to("cuda") pour le GPU
output_ids = model.generate(**input_ids, do_sample=True, top_p=50, temperature=1,max_new_tokens=150) #le ** sert à aplanir le dictionnaire input_ids
#top_p=50 : "à chaque fois qu'il pioche un nouveau mot, il le pioche parmi les 50 mots les plus probables"
#temperature : plus elle est élevée, plus ça accroît la stochasticité (en fait elle divise les logits : = logit/temperature)
#max_new_tokens : règle la longueur de la réponse (qui, par défaut, est de max_length = 20 tokens (incluant ceux de la question tandis qu'
#avec max_new_tokens on compte les tokens de la réponse uniquement
))
print(tokenizer.decode(output_ids[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|>Donne moi une bonne recette de pâtes bolognaises
Pâtes a boire? Non - en espagnol et en italien, «pasta a bevande», c'est plus un dessert. Dans un petit bol, diluez un petit peu de pasta con panna in un bol. (la pasta con panna est une pâte légèrement liquide aux fruits, aux nœuds, aux feuilles ou à la vanille.) Vous pouvez également le faire avec la pâtes bolognese, mais ça devient trop gras. Ou pour un petit dessert, une bonne quantité de cannelloni avec du crème fraîche, du pesto et de l'anis. Ou encore un peu d'emmental et de brocoli bien cuit


## Chargement du modèle Llama 3.2 3B spécialisé pour répondre aux instructions

1. Instancier le tokenizer pour ce modèle
2. Charger le modèle en précision 16 bits, comme le modèle de base

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct",
                                             return_dict=True,
                                             torch_dtype=torch.float16,
                                             device_map="auto")


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

## Génération avec le modèle spécialisé

Le modèle spécialisé nécessite des balises additionnelles, c'est-à-dire des tokens spéciaux, pour distinguer les auteurs des différentres répliques (système, assistant, utilisateur)

1. Formatter l'entrée pour expliciter les auteurs des répliques.
2. Tokenizer l'entrée en ajoutant les balises pour obtenir un texte au format "chat"
3. Générer la réponse

In [ ]:
question= "en quelle année a été créé le laboratoire ERIC ?"
prompt =[
    {"role":"system", "content" : "You are a virtual assistant. Please be helpfull"},
     #définit le rôle qu'on attribue à l'IA - ex : "tu seras un recruteur", "tu seras un assistant",
     #"tu seras un capitaine de bateau pirate", etc
     {"role":"user", "content" : question}
]
chat = tokenizer.apply_chat_template(prompt, tokenize=False)

input_ids = tokenizer(chat, add_special_tokens=True, return_tensors="pt").to("cuda")
output_ids = model.generate(**input_ids, do_sample=True, top_p=50, temperature=1,max_new_tokens=150)
print(tokenizer.decode(output_ids[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 12 Dec 2024

You are a virtual assistant. Please be helpfull<|eot_id|><|start_header_id|>user<|end_header_id|>

en quelle année a été créé le laboratoire ERIC?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Je ne trouve pas d'informations précises sur un laboratoire nommé ERIC. Il y a toutefois plusieurs organisations nommées ERIC et/ou E-RIC en différents domaines :

-   L'ERIC (Education and Resource Information Center) est un centre canadien qui fournit des ressources éducatives à la Communauté française. Il a été créé en 1972.
-   L'ERIC (Educational Resource Information Center) est un centre américain créé dans les années 1970.
-   LEARN (Learning Enhancement and Research Information Network) est un réseau canadien.<|eot_id|>


## Indexation de la base documentaire

1. Dans la cellule ci-dessous, découper le texte en phrases
3. Dans la deuxième cellule, charger l'encodeur siamois pré-entraîné pour le français `dangvantuan/sentence-camembert-base` puis indexer (i.e. vectoriser) les phrases découpées précédemment

In [ ]:
from nltk import download; download('punkt_tab')
import nltk

from nltk.tokenize import sent_tokenize

eric = """Créé en 1995, le laboratoire ERIC (Unité de Recherche des Universités Lyon 2 et Lyon 1) développe des recherches théoriques et appliquées dans les domaines de la science des données et de l’informatique décisionnelle. Elles visent à valoriser les grandes bases de données complexes, notamment dans les domaines des lettres, langues, sciences humaines et sociales (LLSHS) et se situent dans les domaines suivants.
Science des données : machine learning, prévision, décision, fouille de données complexes, textuelles ou fonctionnelles, agrégation multicritère
Informatique décisionnelle : entrepôts de données, intégration intelligente de données complexes, modélisation multidimensionnelle d’objets complexes, analyse en ligne (OLAP) personnalisée, lacs de données, gestion de métadonnées, sécurité des données
Humanités numériques : gestion et analyse de données variées (au sens des mégadonnées), métadonnées, interprétabilité des modèles de fouille de données, informatique et genre
"""
phrases=sent_tokenize(eric)
print(len(phrases))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


3


In [ ]:
!pip install -U bitsandbytes txtai
import txtai

documents=txtai.Embeddings(path="dangvantuan/sentence-camembert-base")
documents.index([(uid,text,None) for uid,text in enumerate(sent_tokenize(eric))])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.3/256.3 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 75.8 MB/s eta 0:00:00


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/463 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

## Recherche et génération

1. Identifier la phrase la plus liée à la question
2. Insérer la phrase dans la partie système du prompt
3. Générer et afficher la réponse

In [ ]:
question ="En quelle année a été créé le laboratoire ERIC ?"
selected_passage = phrases[documents.search(question,1)[0][0]]
prompt =[
    {"role":"system", "content" : "You are a virtual assistant. Please be helpfull"},
     #définit le rôle qu'on attribue à l'IA - ex : "tu seras un recruteur", "tu seras un assistant",
     #"tu seras un capitaine de bateau pirate", etc
     {"role":"user", "content" : question}
]
chat = tokenizer.apply_chat_template(prompt, tokenize=False)

input_ids = tokenizer(chat, add_special_tokens=True, return_tensors="pt").to("cuda")
output_ids = model.generate(**input_ids, do_sample=True, top_p=50, temperature=1,max_new_tokens=150)
print(tokenizer.decode(output_ids[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 12 Dec 2024

You are a virtual assistant. Please be helpfull<|eot_id|><|start_header_id|>user<|end_header_id|>

En quelle année a été créé le laboratoire ERIC?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Je m'appelle ERIC (École nationale supérieure d'ingénierie des technologies de l'information de Lyon). Les Écoles d'ingénieurs de l'institut polytechnique de Grenoble ont créé le laboratoire  en 1973<|eot_id|>
